The below code bases upon the assumption that blick_sitegetter has been executed. The below code will get all data from the defined Blick sites:


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import requests
import pickle
import sys
import re
import csv

In [ ]:
crawled_liste = []

In [ ]:
class Article:
    def __init__(self, title, lead, datum, zeit, author, text, url):
        self.title = title
        self.lead = lead
        self.datum = datum
        self.zeit = zeit
        self.author = author
        self.text = text
        self.url = url

In [ ]:
driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.maximize_window()

In [ ]:
start_seite = "https://www.blick.ch/"
driver.get(start_seite)

cookies = pickle.load(open("blick_cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
    
driver.refresh()

In [ ]:
with open("blick_output.bin", "rb") as data:
    ueb_pages = pickle.load(data)

In [ ]:
def inhalter():    
    driver.get(page)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    
    title = soup.find(class_="title")
    title = title.get_text()

    try:
        lead = soup.find(class_="article-lead")
        lead = lead.get_text()
    except AttributeError:
        lead = ""

    try:
        datum = soup.find(class_=re.compile("ArticleDate__Wrapper"))
        datum = datum.get_text()
        zeit = datum[-10:-3].strip()
        datum = datum[12:-11].strip()
    except AttributeError:
        datum = ""
        zeit = ""

    try:
        author = soup.find(class_=re.compile("detail__item__header__item"))
        author.get_text().strip()
    except AttributeError:
        author = ""

    try:
        text = ""
        texte = soup.find_all("p")
        for x in texte:
            x = x.get_text()
            text += x
    except AttributeError:
        text = ""
        
    durl = page
    
    crawled = Article(title, lead, datum, zeit, author, text, durl)
    crawled_liste.append(crawled)

In [ ]:
for page in ueb_pages[5703:]:
    time.sleep(2)
    try:
        inhalter()
    except TimeoutException:
        print("TimeOut")
        pass
    

In [ ]:
with open('blick_art4.csv', 'w', newline='', encoding="utf8") as csvfile:
    blogwriter = csv.writer(csvfile, delimiter='|', quotechar='"')


    for article in crawled_liste:
        blogwriter.writerow( [article.title, article.lead, article.datum, article.zeit, article.author, article.text, article.url] )

# Quick fix
In case one needs to interrupt crawling and wants to continue later where the crawler stopped, run this cell and update cell 7, first line from: <br>
*"for page in ueb_pages:"* <br>
to : <br>
*"for page in ueb_pages[go_on:]:"*

This method also works in case of uncatched errors or interrupted kernels, provided kernel has not been restarted yet. In case kernel will be restarted, please note down the value of go_on or pickle dump the value beforehand as the value of go_on will not be stored automatically.


In [ ]:
latest = crawled_liste[-1].url    ##get the url from the last object that has been scraped
go_on = ueb_pages.index(latest)   ##find this url in the list that is crawled through and extract its index
print(go_on)